# Recherche de lyrics par artiste

Notre but est de récupérer par artiste, l'ensemble des paroles de ses sons.


__ATTENTION__ ici je ne me concentre que sur les albums des artistes, c'est à dire que tous les song qui ne sont pas sorties dans un album ne sont pas comtabilisé. En effet, sans cette condition difficile de mettre une année sur la diffusion du son.


Nous allons utiliser ce site [https://search.azlyrics.com](https://search.azlyrics.com) pour la recherche des paroles.
<br>
<br>

Voici donc les étapes de la constitution de cette base:

-  Dans un premier temps, la fonction <a href='#artist_to_url'>artist_to_url</a> nous permet de générer à l'aide d'un nom d'artiste un url vers l'ensemble de sa discographie.
-  Notre deuxième fonction <a href='#album_url_year'>album_url_year</a> va nous permettre de ressortir JE NE SAIS PAS ENCORE SOUS QUEL FORMAT le nom de l'artiste, le nom de l'album, l'année de difusion et enfin les liens vers les musiques contenus dans les albums.


In [ ]:
import pandas as pd
import numpy as np
import re
import unidecode
import requests
from bs4 import BeautifulSoup
import urllib
from lxml import html
import sys

from nltk.stem.snowball import EnglishStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import TreebankWordTokenizer

# import spacy

import time

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
#permet de générer l'url pour accéder à l'ensemble des titres de l'artiste
#certain artiste comme Renaud ne sont pas référencé
def artist_to_url(artist):
    """
    
    """
    url_base= """https://www.azlyrics.com"""
    name= re.sub(' ','',artist)
    name= unidecode.unidecode(name.lower()) #me permet d'enlever les accents comme é ou â
    first_l= list(name)[0]
    #certains artistes ont un nom qui commence par un chiffre
    #dans ce cas, la procédure de création de l'url est un peu différent
    if first_l.isalpha():
        url= """{}/{}/{}.html""".format(url_base, first_l, name)
    else:
        url= """{}/19/{}.html""".format(url_base, name)
    return url

In [ ]:
#test
artist_to_url("Fewer 333")

In [ ]:
def album_url_year(artist, timed= True):
    """
    
    """
    year, album_name, url_song= [], [], []
    
    page= requests.get(artist_to_url(artist)) #l'appelle la fonction précédente
    soup= BeautifulSoup(page.text, 'html.parser')
    html_page= soup.find('div', id= 'listAlbum')
    
    if timed:
        time.sleep(np.random.randint(5, 30, 1))
    
    html_data= str(html_page).split('<div class="album"')[1:][:-1]
    for ht in html_data:
    #je défini mes patterns REGEX pour récupérer de la page html le nom des albums, avec son année et les url des lyrics menant aux paroles de chacunes des musiques de l'album     
        pattern_yr= re.compile('</b> \((\d{4})\)</div>|</b> \((\d{4})\)<br/>')
        pattern_alb= re.compile('<b>"([\s\S]*)"</b>')
        pattern_url= re.compile('href="..([\s\S]*).html" target=')

        year.append(pattern_yr.search(ht).group(1))
        album_name.append(pattern_alb.search(ht).group(1))
        #le traitement pour les url est un peu différent
        url_list= ht.split('\n')
        url_base= "https://www.azlyrics.com"
        url_song.append([url_base + pattern_url.search(href).group(1) + ".html" \
                for href in url_list \
                if pattern_url.search(href) != None])
    try:
        df= pd.DataFrame(
        {
            "Artiste": [artist]*len(year),
            "Annee": year,
            "Album": album_name,
            "Url": url_song
        })
    except ValueError as err:
        print("Attention, il semble que les listes du df ne soient \
        pas de la même longueur", err)
        
    #le df que je creer contient en URF une liste des url, je veux donc avoir une ligne par URL et donc
    #les valeurs annee album, et artiste qui correspondent.
    df_not_listed= pd.DataFrame({
      col:np.repeat(df[col].values, df["Url"].str.len())
      for col in df.columns.drop("Url")
    }
    ).assign(**{
        "Url":np.concatenate(df["Url"].values)
    })[df.columns]
    
    return df_not_listed

In [ ]:
album_url_year("jinjer")

In [ ]:
# artiste_name_style= {"Slipknot":"metal"
# ,"I Prevail":"metal"
# ,"DREAMERS":"Rock/Alternatif"
# ,"Bring Me The Horizon":"metal"
# ,"DIAMANTE":"metal"
# ,"Motionless in White":"Rock\Alternatif"
# ,"Demon Hunter":"metal"
# ,"Falling In Reverse":"metal"
# }
artiste_name_style= {"Slipknot":"metal"
,"DREAMERS":"Rock/Alternatif"
,"Bring Me The Horizon":"metal"
,"Motionless in White":"Rock\Alternatif"
,"Falling In Reverse":"metal"
,"Eminem":"rap"
,"Justin Bieber":"pop"
,"Post Malone":"rap"
,"Billie Eilish":"pop"
, "Mac Miller":"rap"
}
artiste_name= [name for name in artiste_name_style.keys()]

df= pd.DataFrame()
for art in artiste_name:
    print(art)
    df= pd.concat([album_url_year(art, False), df], ignore_index= True)

print("\nFin")

In [ ]:
#ajout colonne Style à partir du dictionnaire
df["Style"]= df["Artiste"].map(artiste_name_style)
df.head()

In [ ]:
def verification(df):
    print("Vérifier si des valeurs sont None")
    for col in df.columns:
        nul= (df[col].isnull()).sum()
        sh= df.shape[0]
        print("\n")
        print(col)
        print(nul)
        print("Soit: {}% de valeurs None".format((nul/sh)*100))
    pass

In [ ]:
#on enregistre le dataset contenant les url
export_df= df.to_csv("data/dataset_url_lyrics.csv", index= False, header= True)

## Transformation de nos URL en lyrics

In [ ]:
df= pd.read_csv('data/dataset_url_lyrics.csv')
print(df.shape)
df.head()

In [ ]:
def pretransformation_lyric(url_son):
    """
    
    """
    try:
        #on se connecte et on récupère la partie de la page avec les lyrics
        page= requests.get(url_son)
        soup= BeautifulSoup(page.text, 'html.parser')

        lyric= str(page.content)
        pattern= re.compile('(?:Sorry about that. -->)([\s\S]*)(?:-- MxM)')
        res= pattern.search(lyric).group(1)

        #On va transformer notre texte de telle sorte qu'il soit exploitable par nos Lemmnizer / Stemmenizer
        #J'encode en #!utf8 pour accepter les \n etc
        res= res.encode('utf8').decode("unicode_escape")
        #J'y ai mis une balise que j'ai du mal à supprimer
        banword= ['br', 'div', 'brbr']
        #On tokenize notre text
        tokenizer= TreebankWordTokenizer()
        #Je sépare les élément tokenizer
        tokenz= [','.join(tokenizer.tokenize(mot)) for mot in res.split()]

        tokenz= [mot.replace(",", "").replace("<br>", "") for mot in tokenz]
        #J'enlève ce qui n'est pas du texte ou en espace
        tokenz= [re.sub('[^\w\s]', ' ', mot) for mot in tokenz]
        #Je supprime les espaces inutiles
        tokenz= [mot.replace(' ','') for mot in tokenz]
        #Et enfin j'applique ma liste de banword
    #     text_clean= [mot for mot in tokenz if mot not in banword]
        text_clean= ''
        for mot in tokenz:
            if mot not in banword:
                text_clean += mot + ' '

        #on fait un sleep entre 5 et 30 sec
        time.sleep(np.random.randint(5, 40, 1))

        return text_clean #sortie de cette facon pour utiliser lemma de spacy
        
    except:
        return None

In [ ]:
# liste_lyrics= [pretransformation_lyric(url) \
#               for url in df["Url"].values]
# liste_lyrics

In [ ]:
print(df.shape)
df = df.sample(frac=1).reset_index(drop=True)
export_df= df.to_csv("random_dataset_url_lyrics.csv", index= False, header= True)
df.head()

In [ ]:
df["Artiste"].unique()

In [ ]:
#on va scinder notre dataset pour faire plusieurs set de web scraping
df_1= df.iloc[:200,:]
df_2= df.iloc[200:400,:]
df_3= df.iloc[400:600,:]
df_4= df.iloc[600:800,:]
df_5= df.iloc[800:1000,:]
df_6= df.iloc[1000:df.shape[0],:]

In [ ]:
df_1["test"]= df_1.apply(lambda row: pretransformation_lyric(row[3])
                     ,axis= 1)
#pour gérer les erreurs, tester si la page nous a repéré, si oui ==> capcha ou acces denied ?

In [ ]:
print(df_1.shape)
verification(df_1)

In [ ]:
export_df= df_1.to_csv("data/01_dataset_lyrics.csv", index= False, header= True)

In [ ]:
df_2["test"]= df_2.apply(lambda row: pretransformation_lyric(row[3])
                     ,axis= 1)
#pour gérer les erreurs, tester si la page nous a repéré, si oui ==> capcha ou acces denied ?
print(df_2.shape)
verification(df_2)

In [ ]:
export_df= df_2.to_csv("data/02_dataset_lyrics.csv", index= False, header= True)

In [ ]:
df_3["test"]= df_3.apply(lambda row: pretransformation_lyric(row[3])
                     ,axis= 1)
#pour gérer les erreurs, tester si la page nous a repéré, si oui ==> capcha ou acces denied ?
print(df_3.shape)
verification(df_3)

In [ ]:
export_df= df_3.to_csv("data/03_dataset_lyrics.csv", index= False, header= True)

In [ ]:
df= pd.read_csv("data/random_dataset_url_lyrics.csv")

In [ ]:
df_4= df.iloc[600:800,:]
df_5= df.iloc[800:1000,:]
df_6= df.iloc[1000:df.shape[0],:]

In [ ]:
df_4["test"]= df_4.apply(lambda row: pretransformation_lyric(row[3])
                     ,axis= 1)
#pour gérer les erreurs, tester si la page nous a repéré, si oui ==> capcha ou acces denied ?
print(df_4.shape)
verification(df_4)

In [ ]:
export_df= df_4.to_csv("data/04_dataset_lyrics.csv", index= False, header= True)

In [ ]:
df_5["test"]= df_5.apply(lambda row: pretransformation_lyric(row[3])
                     ,axis= 1)
#pour gérer les erreurs, tester si la page nous a repéré, si oui ==> capcha ou acces denied ?
print(df_5.shape)
verification(df_5)

In [ ]:
export_df= df_5.to_csv("data/05_dataset_lyrics.csv", index= False, header= True)

In [ ]:
df_6["test"]= df_6.apply(lambda row: pretransformation_lyric(row[3])
                     ,axis= 1)
#pour gérer les erreurs, tester si la page nous a repéré, si oui ==> capcha ou acces denied ?
print(df_6.shape)
verification(df_6)

In [ ]:
export_df= df_6.to_csv("data/06_dataset_lyrics.csv", index= False, header= True)